<a href="https://colab.research.google.com/github/Ayush310803/ViT/blob/main/Unettrainipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade tensorflow-datasets

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow_datasets as tfds

builder = tfds.builder('oxford_iiit_pet:4.*.*')

builder.download_and_prepare()

dataset = builder.as_data_source()
info = builder.info
print(info)

tfds.core.DatasetInfo(
    name='oxford_iiit_pet',
    full_name='oxford_iiit_pet/4.0.0',
    description="""
    The Oxford-IIIT pet dataset is a 37 category pet image dataset with roughly 200
    images for each class. The images have large variations in scale, pose and
    lighting. All images have an associated ground truth annotation of breed and
    species. Additionally, head bounding boxes are provided for the training split,
    allowing using this dataset for simple object detection tasks. In the test
    split, the bounding boxes are empty.
    """,
    homepage='http://www.robots.ox.ac.uk/~vgg/data/pets/',
    data_dir='/root/tensorflow_datasets/oxford_iiit_pet/4.0.0',
    file_format=array_record,
    download_size=Unknown size,
    dataset_size=773.68 MiB,
    features=FeaturesDict({
        'file_name': Text(shape=(), dtype=string),
        'head_bbox': BBoxFeature(shape=(4,), dtype=float32),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': Cl

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

def data_source_to_tf_dataset(data_source, split, img_size=128):
    def generator():
        for example in data_source[split]:
            image = example["image"]
            mask = example["segmentation_mask"]

            image = tf.image.resize(image, (img_size, img_size), method="nearest")
            mask = tf.image.resize(mask, (img_size, img_size), method="nearest")

            image = tf.cast(image, tf.float32) / 255.0
            mask = tf.cast(mask, tf.int32)
            mask = tf.squeeze(mask, axis=-1)

            yield image, mask

    output_signature = (
        tf.TensorSpec(shape=(img_size, img_size, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(img_size, img_size), dtype=tf.int32),
    )

    return tf.data.Dataset.from_generator(generator, output_signature=output_signature)

IMG_SIZE = 128
train_dataset = data_source_to_tf_dataset(dataset, split='train', img_size=IMG_SIZE)
test_dataset = data_source_to_tf_dataset(dataset, split='test', img_size=IMG_SIZE)

In [ ]:
BATCH_SIZE = 64
train_batches = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_batches = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
validation_batches = test_dataset.take(128).batch(BATCH_SIZE)
test_batches = test_dataset.skip(128).take(669).batch(BATCH_SIZE)


In [ ]:
def double_conv_block(x, n_filters):
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)

   return x

In [ ]:
def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.3)(p)

   return f, p

In [ ]:
def upsample_block(x, conv_features, n_filters):
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   x = layers.concatenate([x, conv_features])
   x = layers.Dropout(0.3)(x)
   x = double_conv_block(x, n_filters)

   return x

In [ ]:
def build_unet_model(input_shape=(128, 128, 3)):
    inputs = layers.Input(shape=input_shape)

    f1, p1 = downsample_block(inputs, 64)
    f2, p2 = downsample_block(p1, 128)
    f3, p3 = downsample_block(p2, 256)
    f4, p4 = downsample_block(p3, 512)

    bottleneck = double_conv_block(p4, 1024)

    u6 = upsample_block(bottleneck, f4, 512)
    u7 = upsample_block(u6, f3, 256)
    u8 = upsample_block(u7, f2, 128)
    u9 = upsample_block(u8, f1, 64)

    outputs = layers.Conv2D(4, 1, padding="same", activation="softmax")(u9)

    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
    return unet_model

In [ ]:
unet_model = build_unet_model()

In [ ]:
unet_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

In [ ]:
NUM_EPOCHS = 5


TRAIN_LENGTH = info.splits["train"].num_examples
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

VAL_SUBSPLITS = 5
TEST_LENGTH = info.splits["test"].num_examples
VALIDATION_STEPS = TEST_LENGTH // BATCH_SIZE // VAL_SUBSPLITS

model_history = unet_model.fit(
    train_batches,
    epochs=NUM_EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    validation_data=test_batches
)

Epoch 1/5
57/57 ━━━━━━━━━━━━━━━━━━━━ 7938s 139s/step - accuracy: 0.5454 - loss: 1.0147 - val_accuracy: 0.6486 - val_loss: 0.7868
Epoch 2/5
 1/57 ━━━━━━━━━━━━━━━━━━━━ 1:00:56 65s/step - accuracy: 0.6045 - loss: 0.8307

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


In [ ]:
def create_mask(pred_mask):
 pred_mask = tf.argmax(pred_mask, axis=-1)
 pred_mask = pred_mask[..., tf.newaxis]
 return pred_mask[0]

def show_predictions(dataset=None, num=1):
 if dataset:
   for image, mask in dataset.take(num):
     pred_mask = unet_model.predict(image)
     display([image[0], mask[0], create_mask(pred_mask)])
 else:
   display([sample_image, sample_mask,
            create_mask(model.predict(sample_image[tf.newaxis, ...]))])

count = 0
for i in test_batches:
   count +=1
print("number of batches:", count)